In [1]:
import sys, os
import csv
import datetime

from __future__ import division

sys.path.insert(0, '../bifacial_radiance')

try:
    from bifacial_radiance_ModelComparison import *
    from bifacial_radiance import *
    print "MODIFIED Bifacial Radiance Imported!"
except ImportError:
    raise RuntimeError('bifacial_radiance is required. download distribution')

import sys
sys.path.insert(0, '../../bifacialvf/bifacialvf')

from modelPaper import readSRRL
    

MODIFIED Bifacial Radiance Imported!


In [2]:
# Disabling printing inside of Radiance Function because it's too much

import os, sys

class HiddenPrints:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')

    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout = self._original_stdout
        
with HiddenPrints():
    print("This will not be printed")

print("This will be printed as before")

This will be printed as before


In [6]:
#TEST
# Simple example system using Radiance.
#point to an empty directory or existing Radiance directory
now = datetime.datetime.now()
print "DATE", str(now)

writefiletitle="C:\Users\Silvana\Documents\GitHub\\bifacialvf\\bifacialvf\data\Output\Setup1_Rad_test2.csv"
datestart = '2017-08-12 09:40:00'
dateend = '2017-08-12 10:00:00'
rtr=0.9144
C=0.5
beta = 10

testfolder = r'C:\Users\silvana\Documents\RadianceScene\Test2'
print testfolder

SRRLtoread = "C:\Users\Silvana\Documents\GitHub\\bifacialvf\\bifacialvf\data\SRRL_data.csv"
print SRRLtoread
demo = RadianceObj('mini_panel',testfolder)  
demo.setGround(0.62) # input albedo or material name like 'concrete'
#   epwfile = demo.getEPW(37.5,-77.6) #can't run this within NREL firewall.  BOO
metdata = demo.readEPW('EPWs\USA_VA_Richmond.Intl.AP.724010_TMY.epw')
# sky data for index 4010 - 4028 (June 17)  
#demo.gendaylit(metdata,4020)

data = readSRRL(SRRLtoread, decimate = True, decimatestyle = "Mean", decimatevalue = 5 )

# Masking data to only do values we are interested in
mask = (data.index > datestart) & (data.index <= dateend )
data=data.loc[mask]
data=data.dropna()
        
direct_normal = data.loc[:,"Direct"]
diffuse_horizontal = data.loc[:,"Diffuse"]
years=data.loc[:,"Year"]
months=data.loc[:,"Month"]
days=data.loc[:,"Day"]
hours=data.loc[:,"Hour"]
minutes=data.loc[:,"Minute"]
ambient_temperatures=data.loc[:,"Temp"]
wind_speeds=data.loc[:,"Wind"]

noRows=len(direct_normal)         
#    SRRL = ({'Month': month_all, 'Day': day_all, 'Year': year_all, 'Hour': hour_all, 'Minute': minute_all,
#                 'Global': global_all, 'Direct': direct_all, 'Diffuse': diffuse_all, 
#                 'Temp': temp_all, 'Humidity': relhum_all, 'Wind': wind_all})
    
with open (writefiletitle,'w') as csvfile:
    sw = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)

    outputtitles=['Year', 'Month', 'Day', 'Hour', 'Minute', 'DNI', 'DHI', 'Tamb', 'VWind',
                   'IMT1_Rad', 'IMT2_Rad', 'IMT3_Rad' ,'IMT4_Rad', 'IMT5_Rad', 'IMT6_Rad']
    sw.writerow(outputtitles)

    for rl in range (0, len(data)):
    
        hour=hours[rl]
        minute=minutes[rl]
        day=days[rl]
        month=months[rl]
        year=years[rl]

        my_hour=round((float(hour)+float(minute)/60),2)

        dni= direct_normal[rl]
        dhi= diffuse_horizontal[rl]

        if dni<0:
            dni=0
        if dhi<0:
            dhi=0

        try:
            print " HELLO "
            demo.gendaylitSil(month,day, my_hour, dni, dhi)
            #demo.genCumSky(demo.epwfile)
            # create a scene using monopanel in landscape at 10 deg tilt, 1.5m pitch
            sceneDict = {'tilt':beta,'pitch':rtr,'height':C,'orientation':'landscape','azimuth':180}
            scene = demo.makeScene('mini_panel',sceneDict)
            filelist = demo.getfilelist() #filelist now generated in this function
            octfile = demo.makeOct(filelist) 
            analysis = AnalysisObj(octfile, demo.basename)
            OTFDict=analysis.analysisSil(octfile, demo.basename, scene.frontscan, scene.backscan)    

            myfrontlpts=["0 0.0935567365362 0.01 0 0 1\n0 0.187113473072 0.01 0 0 1\n0 0.280670209608 0.01 0 0 1\n0 0.374226946145 0.01 0 0 1\n0 0.467783682681 0.01 0 0 1\n0 0.561340419217 0.01 0 0 1\n0 0.654897155753 0.01 0 0 1\n0 0.748453892289 0.01 0 0 1\n0 0.842010628825 0.01 0 0 1"]
            data_sub = {key:OTFDict[key] for key in ['x', 'y', 'z', 'Wm2', 'mattype']}

            if OTFDict is not None:                
        #                self.Wm2Front = data_sub.pop('Wm2')
        #                data_sub['Wm2Front'] = self.Wm2Front
        #                self.Wm2Back = backR['Wm2']
        #                data_sub['Wm2Back'] = self.Wm2Back
        #                self.backRatio = [x/(y+.001) for x,y in zip(backR['Wm2'],data['Wm2'])] # add 1mW/m2 to avoid dividebyzero
        #                data_sub['Back/FrontRatio'] = self.backRatio
        #                df = pd.DataFrame.from_dict(data_sub)
                outputvalue=[2017, int(month), int(day), int(hour), int(minute), dni, dhi, ambient_temperatures[rl], wind_speeds[rl]]
                outputvalue+=OTFDict['Wm2']
                sw.writerow(outputvalue)
        except:
            print "Warning: sun altitude below zero at month/day " + str(int(month)) + "/" + str(int(day)) + " time " + str(my_hour)
            pass

print "FINISHED FINISHED"


DATE 2018-04-24 16:04:30.281000
C:\Users\silvana\Documents\RadianceScene\Test2
C:\Users\Silvana\Documents\GitHub\bifacialvf\bifacialvf\data\SRRL_data.csv
path = C:\Users\silvana\Documents\RadianceScene\Test2
Reading SRRL File:  C:\Users\Silvana\Documents\GitHub\bifacialvf\bifacialvf\data\SRRL_data.csv
Decimation options:  True
Decimate Style:  Mean  to interval  5  minutes.
Data decimated to 5 min Interval by Average
 HELLO 
 HELLO 
 HELLO 
 HELLO 
FINISHED FINISHED


In [ ]:
#SETUP 1
# Simple example system using Radiance.
#point to an empty directory or existing Radiance directory
writefiletitle="C:\Users\Silvana\Documents\GitHub\\bifacialvf\\bifacialvf\data\Output\Setup1_Rad.csv"
datestart = '2017-08-11 17:38:00'
dateend = '2017-08-24 13:59:00'
rtr=0.9144
C=0.5
beta = 10

testfolder = r'C:\Users\silvana\Documents\RadianceScene\Test'
SRRLtoread = "C:\Users\Silvana\Documents\GitHub\\bifacialvf\\bifacialvf\data\SRRL_data.csv"
demo = RadianceObj('mini_panel',testfolder)  
demo.setGround(0.62) # input albedo or material name like 'concrete'
#   epwfile = demo.getEPW(37.5,-77.6) #can't run this within NREL firewall.  BOO
metdata = demo.readEPW('EPWs\USA_VA_Richmond.Intl.AP.724010_TMY.epw')
# sky data for index 4010 - 4028 (June 17)  
#demo.gendaylit(metdata,4020)

data = readSRRL(SRRLtoread, decimate = True, decimatestyle = "Mean", decimatevalue = 5 )

# Masking data to only do values we are interested in
mask = (data.index > datestart) & (data.index <= dateend )
data=data.loc[mask]
data=data.dropna()
        
direct_normal = data.loc[:,"Direct"]
diffuse_horizontal = data.loc[:,"Diffuse"]
years=data.loc[:,"Year"]
months=data.loc[:,"Month"]
days=data.loc[:,"Day"]
hours=data.loc[:,"Hour"]
minutes=data.loc[:,"Minute"]
ambient_temperatures=data.loc[:,"Temp"]
wind_speeds=data.loc[:,"Wind"]

noRows=len(direct_normal)         
#    SRRL = ({'Month': month_all, 'Day': day_all, 'Year': year_all, 'Hour': hour_all, 'Minute': minute_all,
#                 'Global': global_all, 'Direct': direct_all, 'Diffuse': diffuse_all, 
#                 'Temp': temp_all, 'Humidity': relhum_all, 'Wind': wind_all})
    
with open (writefiletitle,'w') as csvfile:
    sw = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)

    outputtitles=['Year', 'Month', 'Day', 'Hour', 'Minute', 'DNI', 'DHI', 'Tamb', 'VWind',
                   'IMT1_Rad', 'IMT2_Rad', 'IMT3_Rad' ,'IMT4_Rad', 'IMT5_Rad', 'IMT6_Rad']
    sw.writerow(outputtitles)

    for rl in range (0, len(data)):
    
        hour=hours[rl]
        minute=minutes[rl]
        day=days[rl]
        month=months[rl]
        year=years[rl]

        my_hour=round((float(hour)+float(minute)/60),2)

        dni= direct_normal[rl]
        dhi= diffuse_horizontal[rl]

        if dni<0:
            dni=0
        if dhi<0:
            dhi=0

        try:
            with HiddenPrints():
                demo.gendaylitSil(month,day, my_hour, dni, dhi)
                #demo.genCumSky(demo.epwfile)
                # create a scene using monopanel in landscape at 10 deg tilt, 1.5m pitch
                sceneDict = {'tilt':beta,'pitch':rtr,'height':C,'orientation':'landscape','azimuth':180}
                scene = demo.makeScene('mini_panel',sceneDict)
                filelist = demo.getfilelist() #filelist now generated in this function
                octfile = demo.makeOct(filelist) 
                analysis = AnalysisObj(octfile, demo.basename)
                OTFDict=analysis.analysisSil(octfile, demo.basename, scene.frontscan, scene.backscan)    

                myfrontlpts=["0 0.0935567365362 0.01 0 0 1\n0 0.187113473072 0.01 0 0 1\n0 0.280670209608 0.01 0 0 1\n0 0.374226946145 0.01 0 0 1\n0 0.467783682681 0.01 0 0 1\n0 0.561340419217 0.01 0 0 1\n0 0.654897155753 0.01 0 0 1\n0 0.748453892289 0.01 0 0 1\n0 0.842010628825 0.01 0 0 1"]
                data_sub = {key:OTFDict[key] for key in ['x', 'y', 'z', 'Wm2', 'mattype']}

            if OTFDict is not None:                
        #                self.Wm2Front = data_sub.pop('Wm2')
        #                data_sub['Wm2Front'] = self.Wm2Front
        #                self.Wm2Back = backR['Wm2']
        #                data_sub['Wm2Back'] = self.Wm2Back
        #                self.backRatio = [x/(y+.001) for x,y in zip(backR['Wm2'],data['Wm2'])] # add 1mW/m2 to avoid dividebyzero
        #                data_sub['Back/FrontRatio'] = self.backRatio
        #                df = pd.DataFrame.from_dict(data_sub)
                outputvalue=[2017, int(month), int(day), int(hour), int(minute), dni, dhi, ambient_temperatures[rl], wind_speeds[rl]]
                outputvalue+=OTFDict['Wm2']
                sw.writerow(outputvalue)
        except:
            print "Warning: sun altitude below zero at month/day " + str(int(month)) + "/" + str(int(day)) + " time " + str(my_hour)
            pass
print "FINISHED FINISHED"


In [5]:
#SETUP 2
# Simple example system using Radiance.
#point to an empty directory or existing Radiance directory
writefiletitle="C:\Users\Silvana\Documents\GitHub\\bifacialvf\\bifacialvf\data\Output\Setup2_Rad.csv"
datestart = '2017-08-25 10:00:00'  # Sep. 25 10 am STD time (11 Golden) to end of Aug. 31  
dateend = '2017-09-01 12:59:00'
rtr=0.9144
C=0.36576
beta = 10
    
testfolder = r'C:\Users\silvana\Documents\RadianceScene\Test'
SRRLtoread = "C:\Users\Silvana\Documents\GitHub\\bifacialvf\\bifacialvf\data\SRRL_data.csv"
demo = RadianceObj('mini_panel',testfolder)  
demo.setGround(0.62) # input albedo or material name like 'concrete'
#   epwfile = demo.getEPW(37.5,-77.6) #can't run this within NREL firewall.  BOO
metdata = demo.readEPW('EPWs\USA_VA_Richmond.Intl.AP.724010_TMY.epw')
# sky data for index 4010 - 4028 (June 17)  
#demo.gendaylit(metdata,4020)

data = readSRRL(SRRLtoread, decimate = True, decimatestyle = "Mean", decimatevalue = 5 )

# Masking data to only do values we are interested in
mask = (data.index > datestart) & (data.index <= dateend )
data=data.loc[mask]
data=data.dropna()
        
direct_normal = data.loc[:,"Direct"]
diffuse_horizontal = data.loc[:,"Diffuse"]
years=data.loc[:,"Year"]
months=data.loc[:,"Month"]
days=data.loc[:,"Day"]
hours=data.loc[:,"Hour"]
minutes=data.loc[:,"Minute"]
ambient_temperatures=data.loc[:,"Temp"]
wind_speeds=data.loc[:,"Wind"]

noRows=len(direct_normal)         
#    SRRL = ({'Month': month_all, 'Day': day_all, 'Year': year_all, 'Hour': hour_all, 'Minute': minute_all,
#                 'Global': global_all, 'Direct': direct_all, 'Diffuse': diffuse_all, 
#                 'Temp': temp_all, 'Humidity': relhum_all, 'Wind': wind_all})
    
with open (writefiletitle,'w') as csvfile:
    sw = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)

    outputtitles=['Year', 'Month', 'Day', 'Hour', 'Minute', 'DNI', 'DHI', 'Tamb', 'VWind',
                   'IMT1_Rad', 'IMT2_Rad', 'IMT3_Rad' ,'IMT4_Rad', 'IMT5_Rad', 'IMT6_Rad']
    sw.writerow(outputtitles)

    for rl in range (0, len(data)):
    
        hour=hours[rl]
        minute=minutes[rl]
        day=days[rl]
        month=months[rl]
        year=years[rl]

        my_hour=round((float(hour)+float(minute)/60),2)

        dni= direct_normal[rl]
        dhi= diffuse_horizontal[rl]

        if dni<0:
            dni=0
        if dhi<0:
            dhi=0

        try:
            with HiddenPrints():
                demo.gendaylitSil(month,day, my_hour, dni, dhi)
                #demo.genCumSky(demo.epwfile)
                # create a scene using monopanel in landscape at 10 deg tilt, 1.5m pitch
                sceneDict = {'tilt':beta,'pitch':rtr,'height':C,'orientation':'landscape','azimuth':180}
                scene = demo.makeScene('mini_panel',sceneDict)
                filelist = demo.getfilelist() #filelist now generated in this function
                octfile = demo.makeOct(filelist) 
                analysis = AnalysisObj(octfile, demo.basename)
                OTFDict=analysis.analysisSil(octfile, demo.basename, scene.frontscan, scene.backscan)    

                myfrontlpts=["0 0.0935567365362 0.01 0 0 1\n0 0.187113473072 0.01 0 0 1\n0 0.280670209608 0.01 0 0 1\n0 0.374226946145 0.01 0 0 1\n0 0.467783682681 0.01 0 0 1\n0 0.561340419217 0.01 0 0 1\n0 0.654897155753 0.01 0 0 1\n0 0.748453892289 0.01 0 0 1\n0 0.842010628825 0.01 0 0 1"]
                data_sub = {key:OTFDict[key] for key in ['x', 'y', 'z', 'Wm2', 'mattype']}

            if OTFDict is not None:                
        #                self.Wm2Front = data_sub.pop('Wm2')
        #                data_sub['Wm2Front'] = self.Wm2Front
        #                self.Wm2Back = backR['Wm2']
        #                data_sub['Wm2Back'] = self.Wm2Back
        #                self.backRatio = [x/(y+.001) for x,y in zip(backR['Wm2'],data['Wm2'])] # add 1mW/m2 to avoid dividebyzero
        #                data_sub['Back/FrontRatio'] = self.backRatio
        #                df = pd.DataFrame.from_dict(data_sub)
                outputvalue=[2017, int(month), int(day), int(hour), int(minute), dni, dhi, ambient_temperatures[rl], wind_speeds[rl]]
                outputvalue+=OTFDict['Wm2']
                sw.writerow(outputvalue)
        except:
            print "Warning: sun altitude below zero at month/day " + str(int(month)) + "/" + str(int(day)) + " time " + str(my_hour)
            pass

print "FINISHED FINISHED"


path = C:\Users\silvana\Documents\RadianceScene\Test
Reading SRRL File:  C:\Users\Silvana\Documents\GitHub\bifacialvf\bifacialvf\data\SRRL_data.csv
Decimation options:  True
Decimate Style:  Mean  to interval  5  minutes.
Data decimated to 5 min Interval by Average
FINISHED FINISHED


In [6]:
#SETUP 3
# Simple example system using Radiance.
#point to an empty directory or existing Radiance directory
writefiletitle="C:\Users\Silvana\Documents\GitHub\\bifacialvf\\bifacialvf\data\Output\Setup3_Rad.csv"
datestart = '2017-09-02 05:21:00'  
dateend = '2017-09-11 11:59:00'    # 3)Sep. 11 before 12:15 STD time 
rtr=0.9144
C=0.09144
beta = 10
    
testfolder = r'C:\Users\silvana\Documents\RadianceScene\Test'
SRRLtoread = "C:\Users\Silvana\Documents\GitHub\\bifacialvf\\bifacialvf\data\SRRL_data.csv"
demo = RadianceObj('mini_panel',testfolder)  
demo.setGround(0.62) # input albedo or material name like 'concrete'
#   epwfile = demo.getEPW(37.5,-77.6) #can't run this within NREL firewall.  BOO
metdata = demo.readEPW('EPWs\USA_VA_Richmond.Intl.AP.724010_TMY.epw')
# sky data for index 4010 - 4028 (June 17)  
#demo.gendaylit(metdata,4020)

data = readSRRL(SRRLtoread, decimate = True, decimatestyle = "Mean", decimatevalue = 5 )

# Masking data to only do values we are interested in
mask = (data.index > datestart) & (data.index <= dateend )
data=data.loc[mask]
data=data.dropna()
        
direct_normal = data.loc[:,"Direct"]
diffuse_horizontal = data.loc[:,"Diffuse"]
years=data.loc[:,"Year"]
months=data.loc[:,"Month"]
days=data.loc[:,"Day"]
hours=data.loc[:,"Hour"]
minutes=data.loc[:,"Minute"]
ambient_temperatures=data.loc[:,"Temp"]
wind_speeds=data.loc[:,"Wind"]

noRows=len(direct_normal)         
#    SRRL = ({'Month': month_all, 'Day': day_all, 'Year': year_all, 'Hour': hour_all, 'Minute': minute_all,
#                 'Global': global_all, 'Direct': direct_all, 'Diffuse': diffuse_all, 
#                 'Temp': temp_all, 'Humidity': relhum_all, 'Wind': wind_all})
    
with open (writefiletitle,'w') as csvfile:
    sw = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)

    outputtitles=['Year', 'Month', 'Day', 'Hour', 'Minute', 'DNI', 'DHI', 'Tamb', 'VWind',
                   'IMT1_Rad', 'IMT2_Rad', 'IMT3_Rad' ,'IMT4_Rad', 'IMT5_Rad', 'IMT6_Rad']
    sw.writerow(outputtitles)

    for rl in range (0, len(data)):
    
        hour=hours[rl]
        minute=minutes[rl]
        day=days[rl]
        month=months[rl]
        year=years[rl]

        my_hour=round((float(hour)+float(minute)/60),2)

        dni= direct_normal[rl]
        dhi= diffuse_horizontal[rl]

        if dni<0:
            dni=0
        if dhi<0:
            dhi=0

        try:
            with HiddenPrints():
                demo.gendaylitSil(month,day, my_hour, dni, dhi)
                #demo.genCumSky(demo.epwfile)
                # create a scene using monopanel in landscape at 10 deg tilt, 1.5m pitch
                sceneDict = {'tilt':beta,'pitch':rtr,'height':C,'orientation':'landscape','azimuth':180}
                scene = demo.makeScene('mini_panel',sceneDict)
                filelist = demo.getfilelist() #filelist now generated in this function
                octfile = demo.makeOct(filelist) 
                analysis = AnalysisObj(octfile, demo.basename)
                OTFDict=analysis.analysisSil(octfile, demo.basename, scene.frontscan, scene.backscan)    

                myfrontlpts=["0 0.0935567365362 0.01 0 0 1\n0 0.187113473072 0.01 0 0 1\n0 0.280670209608 0.01 0 0 1\n0 0.374226946145 0.01 0 0 1\n0 0.467783682681 0.01 0 0 1\n0 0.561340419217 0.01 0 0 1\n0 0.654897155753 0.01 0 0 1\n0 0.748453892289 0.01 0 0 1\n0 0.842010628825 0.01 0 0 1"]
                data_sub = {key:OTFDict[key] for key in ['x', 'y', 'z', 'Wm2', 'mattype']}

            if OTFDict is not None:                
        #                self.Wm2Front = data_sub.pop('Wm2')
        #                data_sub['Wm2Front'] = self.Wm2Front
        #                self.Wm2Back = backR['Wm2']
        #                data_sub['Wm2Back'] = self.Wm2Back
        #                self.backRatio = [x/(y+.001) for x,y in zip(backR['Wm2'],data['Wm2'])] # add 1mW/m2 to avoid dividebyzero
        #                data_sub['Back/FrontRatio'] = self.backRatio
        #                df = pd.DataFrame.from_dict(data_sub)
                outputvalue=[2017, int(month), int(day), int(hour), int(minute), dni, dhi, ambient_temperatures[rl], wind_speeds[rl]]
                outputvalue+=OTFDict['Wm2']
                sw.writerow(outputvalue)
        except:
            print "Warning: sun altitude below zero at month/day " + str(int(month)) + "/" + str(int(day)) + " time " + str(my_hour)
            pass

print "FINISHED FINISHED"


path = C:\Users\silvana\Documents\RadianceScene\Test
Reading SRRL File:  C:\Users\Silvana\Documents\GitHub\bifacialvf\bifacialvf\data\SRRL_data.csv
Decimation options:  True
Decimate Style:  Mean  to interval  5  minutes.
Data decimated to 5 min Interval by Average
FINISHED FINISHED


In [7]:
#SETUP 4
# Simple example system using Radiance.
#point to an empty directory or existing Radiance directory
writefiletitle="C:\Users\Silvana\Documents\GitHub\\bifacialvf\\bifacialvf\data\Output\Setup4_Rad.csv"
datestart = '2017-09-11 13:00:00'   # 4) Separation "2 m" (4 foot on Reacl)  to...
dateend = '2017-10-3 10:59:00'    #   MAke sure this is the end
rtr=1.2192
C=0.09144
beta = 10
    
testfolder = r'C:\Users\silvana\Documents\RadianceScene\Test'
SRRLtoread = "C:\Users\Silvana\Documents\GitHub\\bifacialvf\\bifacialvf\data\SRRL_data.csv"
demo = RadianceObj('mini_panel',testfolder)  
demo.setGround(0.62) # input albedo or material name like 'concrete'
#   epwfile = demo.getEPW(37.5,-77.6) #can't run this within NREL firewall.  BOO
metdata = demo.readEPW('EPWs\USA_VA_Richmond.Intl.AP.724010_TMY.epw')
# sky data for index 4010 - 4028 (June 17)  
#demo.gendaylit(metdata,4020)

data = readSRRL(SRRLtoread, decimate = True, decimatestyle = "Mean", decimatevalue = 5 )

# Masking data to only do values we are interested in
mask = (data.index > datestart) & (data.index <= dateend )
data=data.loc[mask]
data=data.dropna()
        
direct_normal = data.loc[:,"Direct"]
diffuse_horizontal = data.loc[:,"Diffuse"]
years=data.loc[:,"Year"]
months=data.loc[:,"Month"]
days=data.loc[:,"Day"]
hours=data.loc[:,"Hour"]
minutes=data.loc[:,"Minute"]
ambient_temperatures=data.loc[:,"Temp"]
wind_speeds=data.loc[:,"Wind"]

noRows=len(direct_normal)         
#    SRRL = ({'Month': month_all, 'Day': day_all, 'Year': year_all, 'Hour': hour_all, 'Minute': minute_all,
#                 'Global': global_all, 'Direct': direct_all, 'Diffuse': diffuse_all, 
#                 'Temp': temp_all, 'Humidity': relhum_all, 'Wind': wind_all})
    
with open (writefiletitle,'w') as csvfile:
    sw = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)

    outputtitles=['Year', 'Month', 'Day', 'Hour', 'Minute', 'DNI', 'DHI', 'Tamb', 'VWind',
                   'IMT1_Rad', 'IMT2_Rad', 'IMT3_Rad' ,'IMT4_Rad', 'IMT5_Rad', 'IMT6_Rad']
    sw.writerow(outputtitles)

    for rl in range (0, len(data)):
    
        hour=hours[rl]
        minute=minutes[rl]
        day=days[rl]
        month=months[rl]
        year=years[rl]

        my_hour=round((float(hour)+float(minute)/60),2)

        dni= direct_normal[rl]
        dhi= diffuse_horizontal[rl]

        if dni<0:
            dni=0
        if dhi<0:
            dhi=0

        try:
            with HiddenPrints():
                demo.gendaylitSil(month,day, my_hour, dni, dhi)
                #demo.genCumSky(demo.epwfile)
                # create a scene using monopanel in landscape at 10 deg tilt, 1.5m pitch
                sceneDict = {'tilt':beta,'pitch':rtr,'height':C,'orientation':'landscape','azimuth':180}
                scene = demo.makeScene('mini_panel',sceneDict)
                filelist = demo.getfilelist() #filelist now generated in this function
                octfile = demo.makeOct(filelist) 
                analysis = AnalysisObj(octfile, demo.basename)
                OTFDict=analysis.analysisSil(octfile, demo.basename, scene.frontscan, scene.backscan)    

                myfrontlpts=["0 0.0935567365362 0.01 0 0 1\n0 0.187113473072 0.01 0 0 1\n0 0.280670209608 0.01 0 0 1\n0 0.374226946145 0.01 0 0 1\n0 0.467783682681 0.01 0 0 1\n0 0.561340419217 0.01 0 0 1\n0 0.654897155753 0.01 0 0 1\n0 0.748453892289 0.01 0 0 1\n0 0.842010628825 0.01 0 0 1"]
                data_sub = {key:OTFDict[key] for key in ['x', 'y', 'z', 'Wm2', 'mattype']}

            if OTFDict is not None:                
        #                self.Wm2Front = data_sub.pop('Wm2')
        #                data_sub['Wm2Front'] = self.Wm2Front
        #                self.Wm2Back = backR['Wm2']
        #                data_sub['Wm2Back'] = self.Wm2Back
        #                self.backRatio = [x/(y+.001) for x,y in zip(backR['Wm2'],data['Wm2'])] # add 1mW/m2 to avoid dividebyzero
        #                data_sub['Back/FrontRatio'] = self.backRatio
        #                df = pd.DataFrame.from_dict(data_sub)
                outputvalue=[2017, int(month), int(day), int(hour), int(minute), dni, dhi, ambient_temperatures[rl], wind_speeds[rl]]
                outputvalue+=OTFDict['Wm2']
                sw.writerow(outputvalue)
        except:
            print "Warning: sun altitude below zero at month/day " + str(int(month)) + "/" + str(int(day)) + " time " + str(my_hour)
            pass

print "FINISHED FINISHED"


path = C:\Users\silvana\Documents\RadianceScene\Test
Reading SRRL File:  C:\Users\Silvana\Documents\GitHub\bifacialvf\bifacialvf\data\SRRL_data.csv
Decimation options:  True
Decimate Style:  Mean  to interval  5  minutes.
Data decimated to 5 min Interval by Average
FINISHED FINISHED


In [8]:
#SETUP 5
# Simple example system using Radiance.
#point to an empty directory or existing Radiance directory
writefiletitle="C:\Users\Silvana\Documents\GitHub\\bifacialvf\\bifacialvf\data\Output\Setup5_Rad.csv"
datestart = '2017-10-03 11:00:00'   # 
dateend = '2017-10-08 19:00:00'    #  Ignoring data from 10/9 and 10/10 from SNOW
rtr=1.8288
C=0.09144
beta = 10
    
testfolder = r'C:\Users\silvana\Documents\RadianceScene\Test'
SRRLtoread = "C:\Users\Silvana\Documents\GitHub\\bifacialvf\\bifacialvf\data\SRRL_data.csv"
demo = RadianceObj('mini_panel',testfolder)  
demo.setGround(0.62) # input albedo or material name like 'concrete'
#   epwfile = demo.getEPW(37.5,-77.6) #can't run this within NREL firewall.  BOO
metdata = demo.readEPW('EPWs\USA_VA_Richmond.Intl.AP.724010_TMY.epw')
# sky data for index 4010 - 4028 (June 17)  
#demo.gendaylit(metdata,4020)

data = readSRRL(SRRLtoread, decimate = True, decimatestyle = "Mean", decimatevalue = 5 )

# Masking data to only do values we are interested in
mask = (data.index > datestart) & (data.index <= dateend )
data=data.loc[mask]
data=data.dropna()
        
direct_normal = data.loc[:,"Direct"]
diffuse_horizontal = data.loc[:,"Diffuse"]
years=data.loc[:,"Year"]
months=data.loc[:,"Month"]
days=data.loc[:,"Day"]
hours=data.loc[:,"Hour"]
minutes=data.loc[:,"Minute"]
ambient_temperatures=data.loc[:,"Temp"]
wind_speeds=data.loc[:,"Wind"]

noRows=len(direct_normal)         
#    SRRL = ({'Month': month_all, 'Day': day_all, 'Year': year_all, 'Hour': hour_all, 'Minute': minute_all,
#                 'Global': global_all, 'Direct': direct_all, 'Diffuse': diffuse_all, 
#                 'Temp': temp_all, 'Humidity': relhum_all, 'Wind': wind_all})
    
with open (writefiletitle,'w') as csvfile:
    sw = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)

    outputtitles=['Year', 'Month', 'Day', 'Hour', 'Minute', 'DNI', 'DHI', 'Tamb', 'VWind',
                   'IMT1_Rad', 'IMT2_Rad', 'IMT3_Rad' ,'IMT4_Rad', 'IMT5_Rad', 'IMT6_Rad']
    sw.writerow(outputtitles)

    for rl in range (0, len(data)):
    
        hour=hours[rl]
        minute=minutes[rl]
        day=days[rl]
        month=months[rl]
        year=years[rl]

        my_hour=round((float(hour)+float(minute)/60),2)

        dni= direct_normal[rl]
        dhi= diffuse_horizontal[rl]

        if dni<0:
            dni=0
        if dhi<0:
            dhi=0

        try:
            with HiddenPrints():
                demo.gendaylitSil(month,day, my_hour, dni, dhi)
                #demo.genCumSky(demo.epwfile)
                # create a scene using monopanel in landscape at 10 deg tilt, 1.5m pitch
                sceneDict = {'tilt':beta,'pitch':rtr,'height':C,'orientation':'landscape','azimuth':180}
                scene = demo.makeScene('mini_panel',sceneDict)
                filelist = demo.getfilelist() #filelist now generated in this function
                octfile = demo.makeOct(filelist) 
                analysis = AnalysisObj(octfile, demo.basename)
                OTFDict=analysis.analysisSil(octfile, demo.basename, scene.frontscan, scene.backscan)    

                myfrontlpts=["0 0.0935567365362 0.01 0 0 1\n0 0.187113473072 0.01 0 0 1\n0 0.280670209608 0.01 0 0 1\n0 0.374226946145 0.01 0 0 1\n0 0.467783682681 0.01 0 0 1\n0 0.561340419217 0.01 0 0 1\n0 0.654897155753 0.01 0 0 1\n0 0.748453892289 0.01 0 0 1\n0 0.842010628825 0.01 0 0 1"]
                data_sub = {key:OTFDict[key] for key in ['x', 'y', 'z', 'Wm2', 'mattype']}

            if OTFDict is not None:                
        #                self.Wm2Front = data_sub.pop('Wm2')
        #                data_sub['Wm2Front'] = self.Wm2Front
        #                self.Wm2Back = backR['Wm2']
        #                data_sub['Wm2Back'] = self.Wm2Back
        #                self.backRatio = [x/(y+.001) for x,y in zip(backR['Wm2'],data['Wm2'])] # add 1mW/m2 to avoid dividebyzero
        #                data_sub['Back/FrontRatio'] = self.backRatio
        #                df = pd.DataFrame.from_dict(data_sub)
                outputvalue=[2017, int(month), int(day), int(hour), int(minute), dni, dhi, ambient_temperatures[rl], wind_speeds[rl]]
                outputvalue+=OTFDict['Wm2']
                sw.writerow(outputvalue)
        except:
            print "Warning: sun altitude below zero at month/day " + str(int(month)) + "/" + str(int(day)) + " time " + str(my_hour)
            pass

print "FINISHED FINISHED"


path = C:\Users\silvana\Documents\RadianceScene\Test
Reading SRRL File:  C:\Users\Silvana\Documents\GitHub\bifacialvf\bifacialvf\data\SRRL_data.csv
Decimation options:  True
Decimate Style:  Mean  to interval  5  minutes.
Data decimated to 5 min Interval by Average
FINISHED FINISHED


In [7]:
#SETUP 6
# Simple example system using Radiance.
#point to an empty directory or existing Radiance directory

now = datetime.datetime.now()
print "DATE OF LAST RUN:", str(now)

writefiletitle="C:\Users\Silvana\Documents\GitHub\\bifacialvf\\bifacialvf\data\Output\Setup6_Rad_Albedo0p56.csv"
datestart = '2017-10-18 15:03:00'   # 
dateend = '2017-10-30 8:39:00'    #  Ignoring data from 10/9 and 10/10 from SNOW
rtr=1.8288
C=0.24384
beta = 10
    
testfolder = r'C:\Users\silvana\Documents\RadianceScene\Test'
SRRLtoread = "C:\Users\Silvana\Documents\GitHub\\bifacialvf\\bifacialvf\data\SRRL_data.csv"
demo = RadianceObj('mini_panel',testfolder)  
demo.setGround(0.56) # input albedo or material name like 'concrete'
#   epwfile = demo.getEPW(37.5,-77.6) #can't run this within NREL firewall.  BOO
metdata = demo.readEPW('EPWs\USA_VA_Richmond.Intl.AP.724010_TMY.epw')
# sky data for index 4010 - 4028 (June 17)  
#demo.gendaylit(metdata,4020)

data = readSRRL(SRRLtoread, decimate = True, decimatestyle = "Mean", decimatevalue = 5 )

# Masking data to only do values we are interested in
mask = (data.index > datestart) & (data.index <= dateend )
data=data.loc[mask]
data=data.dropna()
        
direct_normal = data.loc[:,"Direct"]
diffuse_horizontal = data.loc[:,"Diffuse"]
years=data.loc[:,"Year"]
months=data.loc[:,"Month"]
days=data.loc[:,"Day"]
hours=data.loc[:,"Hour"]
minutes=data.loc[:,"Minute"]
ambient_temperatures=data.loc[:,"Temp"]
wind_speeds=data.loc[:,"Wind"]

noRows=len(direct_normal)         
#    SRRL = ({'Month': month_all, 'Day': day_all, 'Year': year_all, 'Hour': hour_all, 'Minute': minute_all,
#                 'Global': global_all, 'Direct': direct_all, 'Diffuse': diffuse_all, 
#                 'Temp': temp_all, 'Humidity': relhum_all, 'Wind': wind_all})
    
with open (writefiletitle,'w') as csvfile:
    sw = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)

    outputtitles=['Year', 'Month', 'Day', 'Hour', 'Minute', 'DNI', 'DHI', 'Tamb', 'VWind',
                   'IMT1_Rad', 'IMT2_Rad', 'IMT3_Rad' ,'IMT4_Rad', 'IMT5_Rad', 'IMT6_Rad']
    sw.writerow(outputtitles)

    for rl in range (0, len(data)):
    
        hour=hours[rl]
        minute=minutes[rl]
        day=days[rl]
        month=months[rl]
        year=years[rl]

        my_hour=round((float(hour)+float(minute)/60),2)

        dni= direct_normal[rl]
        dhi= diffuse_horizontal[rl]

        if dni<0:
            dni=0
        if dhi<0:
            dhi=0

        try:
            with HiddenPrints():
                demo.gendaylitSil(month,day, my_hour, dni, dhi)
                #demo.genCumSky(demo.epwfile)
                # create a scene using monopanel in landscape at 10 deg tilt, 1.5m pitch
                sceneDict = {'tilt':beta,'pitch':rtr,'height':C,'orientation':'landscape','azimuth':180}
                scene = demo.makeScene('mini_panel',sceneDict)
                filelist = demo.getfilelist() #filelist now generated in this function
                octfile = demo.makeOct(filelist) 
                analysis = AnalysisObj(octfile, demo.basename)
                OTFDict=analysis.analysisSil(octfile, demo.basename, scene.frontscan, scene.backscan)    

                myfrontlpts=["0 0.0935567365362 0.01 0 0 1\n0 0.187113473072 0.01 0 0 1\n0 0.280670209608 0.01 0 0 1\n0 0.374226946145 0.01 0 0 1\n0 0.467783682681 0.01 0 0 1\n0 0.561340419217 0.01 0 0 1\n0 0.654897155753 0.01 0 0 1\n0 0.748453892289 0.01 0 0 1\n0 0.842010628825 0.01 0 0 1"]
                data_sub = {key:OTFDict[key] for key in ['x', 'y', 'z', 'Wm2', 'mattype']}

            if OTFDict is not None:                
        #                self.Wm2Front = data_sub.pop('Wm2')
        #                data_sub['Wm2Front'] = self.Wm2Front
        #                self.Wm2Back = backR['Wm2']
        #                data_sub['Wm2Back'] = self.Wm2Back
        #                self.backRatio = [x/(y+.001) for x,y in zip(backR['Wm2'],data['Wm2'])] # add 1mW/m2 to avoid dividebyzero
        #                data_sub['Back/FrontRatio'] = self.backRatio
        #                df = pd.DataFrame.from_dict(data_sub)
                outputvalue=[2017, int(month), int(day), int(hour), int(minute), dni, dhi, ambient_temperatures[rl], wind_speeds[rl]]
                outputvalue+=OTFDict['Wm2']
                sw.writerow(outputvalue)
        except:
            print "Warning: sun altitude below zero at month/day " + str(int(month)) + "/" + str(int(day)) + " time " + str(my_hour)
            pass

print "FINISHED FINISHED"


DATE OF LAST RUN: 2018-04-24 15:17:40.329000
path = C:\Users\silvana\Documents\RadianceScene\Test
Reading SRRL File:  C:\Users\Silvana\Documents\GitHub\bifacialvf\bifacialvf\data\SRRL_data.csv
Decimation options:  True
Decimate Style:  Mean  to interval  5  minutes.
Data decimated to 5 min Interval by Average


FINISHED FINISHED


In [4]:
#SETUP 7
# Simple example system using Radiance.
#point to an empty directory or existing Radiance directory
now = datetime.datetime.now()
print str(now)

writefiletitle="C:\Users\Silvana\Documents\GitHub\\bifacialvf\\bifacialvf\data\Output\Setup7_Rad_Albedo0p56.csv"
datestart = '2017-10-30 08:49:00'   # 
dateend = '2017-11-13 14:26:00'    #  Ignoring data from 10/9 and 10/10 from SNOW
rtr=1.2192
C=0.24384
beta = 10
    
testfolder = r'C:\Users\silvana\Documents\RadianceScene\Test'
SRRLtoread = "C:\Users\Silvana\Documents\GitHub\\bifacialvf\\bifacialvf\data\SRRL_data.csv"
demo = RadianceObj('mini_panel',testfolder)  
demo.setGround(0.56) # input albedo or material name like 'concrete'
#   epwfile = demo.getEPW(37.5,-77.6) #can't run this within NREL firewall.  BOO
metdata = demo.readEPW('EPWs\USA_VA_Richmond.Intl.AP.724010_TMY.epw')
# sky data for index 4010 - 4028 (June 17)  
#demo.gendaylit(metdata,4020)

data = readSRRL(SRRLtoread, decimate = True, decimatestyle = "Mean", decimatevalue = 5 )

# Masking data to only do values we are interested in
mask = (data.index > datestart) & (data.index <= dateend )
data=data.loc[mask]
data=data.dropna()
        
direct_normal = data.loc[:,"Direct"]
diffuse_horizontal = data.loc[:,"Diffuse"]
years=data.loc[:,"Year"]
months=data.loc[:,"Month"]
days=data.loc[:,"Day"]
hours=data.loc[:,"Hour"]
minutes=data.loc[:,"Minute"]
ambient_temperatures=data.loc[:,"Temp"]
wind_speeds=data.loc[:,"Wind"]

noRows=len(direct_normal)         
#    SRRL = ({'Month': month_all, 'Day': day_all, 'Year': year_all, 'Hour': hour_all, 'Minute': minute_all,
#                 'Global': global_all, 'Direct': direct_all, 'Diffuse': diffuse_all, 
#                 'Temp': temp_all, 'Humidity': relhum_all, 'Wind': wind_all})
    
with open (writefiletitle,'w') as csvfile:
    sw = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)

    outputtitles=['Year', 'Month', 'Day', 'Hour', 'Minute', 'DNI', 'DHI', 'Tamb', 'VWind',
                   'IMT1_Rad', 'IMT2_Rad', 'IMT3_Rad' ,'IMT4_Rad', 'IMT5_Rad', 'IMT6_Rad']
    sw.writerow(outputtitles)

    for rl in range (0, len(data)):
    
        hour=hours[rl]
        minute=minutes[rl]
        day=days[rl]
        month=months[rl]
        year=years[rl]

        my_hour=round((float(hour)+float(minute)/60),2)

        dni= direct_normal[rl]
        dhi= diffuse_horizontal[rl]

        if dni<0:
            dni=0
        if dhi<0:
            dhi=0

        try:
            with HiddenPrints():
                demo.gendaylitSil(month,day, my_hour, dni, dhi)
                #demo.genCumSky(demo.epwfile)
                # create a scene using monopanel in landscape at 10 deg tilt, 1.5m pitch
                sceneDict = {'tilt':beta,'pitch':rtr,'height':C,'orientation':'landscape','azimuth':180}
                scene = demo.makeScene('mini_panel',sceneDict)
                filelist = demo.getfilelist() #filelist now generated in this function
                octfile = demo.makeOct(filelist) 
                analysis = AnalysisObj(octfile, demo.basename)
                OTFDict=analysis.analysisSil(octfile, demo.basename, scene.frontscan, scene.backscan)    

                myfrontlpts=["0 0.0935567365362 0.01 0 0 1\n0 0.187113473072 0.01 0 0 1\n0 0.280670209608 0.01 0 0 1\n0 0.374226946145 0.01 0 0 1\n0 0.467783682681 0.01 0 0 1\n0 0.561340419217 0.01 0 0 1\n0 0.654897155753 0.01 0 0 1\n0 0.748453892289 0.01 0 0 1\n0 0.842010628825 0.01 0 0 1"]
                data_sub = {key:OTFDict[key] for key in ['x', 'y', 'z', 'Wm2', 'mattype']}

            if OTFDict is not None:                
        #                self.Wm2Front = data_sub.pop('Wm2')
        #                data_sub['Wm2Front'] = self.Wm2Front
        #                self.Wm2Back = backR['Wm2']
        #                data_sub['Wm2Back'] = self.Wm2Back
        #                self.backRatio = [x/(y+.001) for x,y in zip(backR['Wm2'],data['Wm2'])] # add 1mW/m2 to avoid dividebyzero
        #                data_sub['Back/FrontRatio'] = self.backRatio
        #                df = pd.DataFrame.from_dict(data_sub)
                outputvalue=[2017, int(month), int(day), int(hour), int(minute), dni, dhi, ambient_temperatures[rl], wind_speeds[rl]]
                outputvalue+=OTFDict['Wm2']
                sw.writerow(outputvalue)
        except:
            print "Warning: sun altitude below zero at month/day " + str(int(month)) + "/" + str(int(day)) + " time " + str(my_hour)
            pass

print "FINISHED FINISHED"


2018-04-17 16:07:00.055000
path = C:\Users\silvana\Documents\RadianceScene\Test
Reading SRRL File:  C:\Users\Silvana\Documents\GitHub\bifacialvf\bifacialvf\data\SRRL_data.csv
Decimation options:  True
Decimate Style:  Mean  to interval  5  minutes.
Data decimated to 5 min Interval by Average


FINISHED FINISHED


In [ ]:
#SETUP 8
# Simple example system using Radiance.
#point to an empty directory or existing Radiance directory
now = datetime.datetime.now()
print str(now)

writefiletitle="C:\Users\Silvana\Documents\GitHub\\bifacialvf\\bifacialvf\data\Output\Setup8_Rad_Albedo0p56.csv"
datestart = '2017-11-13 14:27:00'   # 
dateend = '2017-11-29 9:49:00'    #  Ignoring data from 10/9 and 10/10 from SNOW
dateend = '2017-12-8 9:00:00'    #  Ignoring data from 10/9 and 10/10 from SNOW
rtr=0.9144
C=0.24384
beta = 10
    
testfolder = r'C:\Users\silvana\Documents\RadianceScene\Test'
SRRLtoread = "C:\Users\Silvana\Documents\GitHub\\bifacialvf\\bifacialvf\data\SRRL_data.csv"
demo = RadianceObj('mini_panel',testfolder)  
demo.setGround(0.56) # input albedo or material name like 'concrete'
#   epwfile = demo.getEPW(37.5,-77.6) #can't run this within NREL firewall.  BOO
metdata = demo.readEPW('EPWs\USA_VA_Richmond.Intl.AP.724010_TMY.epw')
# sky data for index 4010 - 4028 (June 17)  
#demo.gendaylit(metdata,4020)

data = readSRRL(SRRLtoread, decimate = True, decimatestyle = "Mean", decimatevalue = 5 )

# Masking data to only do values we are interested in
mask = (data.index > datestart) & (data.index <= dateend )
data=data.loc[mask]
data=data.dropna()
        
direct_normal = data.loc[:,"Direct"]
diffuse_horizontal = data.loc[:,"Diffuse"]
years=data.loc[:,"Year"]
months=data.loc[:,"Month"]
days=data.loc[:,"Day"]
hours=data.loc[:,"Hour"]
minutes=data.loc[:,"Minute"]
ambient_temperatures=data.loc[:,"Temp"]
wind_speeds=data.loc[:,"Wind"]

noRows=len(direct_normal)         
#    SRRL = ({'Month': month_all, 'Day': day_all, 'Year': year_all, 'Hour': hour_all, 'Minute': minute_all,
#                 'Global': global_all, 'Direct': direct_all, 'Diffuse': diffuse_all, 
#                 'Temp': temp_all, 'Humidity': relhum_all, 'Wind': wind_all})
    
with open (writefiletitle,'w') as csvfile:
    sw = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)

    outputtitles=['Year', 'Month', 'Day', 'Hour', 'Minute', 'DNI', 'DHI', 'Tamb', 'VWind',
                   'IMT1_Rad', 'IMT2_Rad', 'IMT3_Rad' ,'IMT4_Rad', 'IMT5_Rad', 'IMT6_Rad']
    sw.writerow(outputtitles)

    for rl in range (0, len(data)):
    
        hour=hours[rl]
        minute=minutes[rl]
        day=days[rl]
        month=months[rl]
        year=years[rl]

        my_hour=round((float(hour)+float(minute)/60),2)

        dni= direct_normal[rl]
        dhi= diffuse_horizontal[rl]

        if dni<0:
            dni=0
        if dhi<0:
            dhi=0

        try:
            with HiddenPrints():
                demo.gendaylitSil(month,day, my_hour, dni, dhi)
                #demo.genCumSky(demo.epwfile)
                # create a scene using monopanel in landscape at 10 deg tilt, 1.5m pitch
                sceneDict = {'tilt':beta,'pitch':rtr,'height':C,'orientation':'landscape','azimuth':180}
                scene = demo.makeScene('mini_panel',sceneDict)
                filelist = demo.getfilelist() #filelist now generated in this function
                octfile = demo.makeOct(filelist) 
                analysis = AnalysisObj(octfile, demo.basename)
                OTFDict=analysis.analysisSil(octfile, demo.basename, scene.frontscan, scene.backscan)    

                myfrontlpts=["0 0.0935567365362 0.01 0 0 1\n0 0.187113473072 0.01 0 0 1\n0 0.280670209608 0.01 0 0 1\n0 0.374226946145 0.01 0 0 1\n0 0.467783682681 0.01 0 0 1\n0 0.561340419217 0.01 0 0 1\n0 0.654897155753 0.01 0 0 1\n0 0.748453892289 0.01 0 0 1\n0 0.842010628825 0.01 0 0 1"]
                data_sub = {key:OTFDict[key] for key in ['x', 'y', 'z', 'Wm2', 'mattype']}

            if OTFDict is not None:                
        #                self.Wm2Front = data_sub.pop('Wm2')
        #                data_sub['Wm2Front'] = self.Wm2Front
        #                self.Wm2Back = backR['Wm2']
        #                data_sub['Wm2Back'] = self.Wm2Back
        #                self.backRatio = [x/(y+.001) for x,y in zip(backR['Wm2'],data['Wm2'])] # add 1mW/m2 to avoid dividebyzero
        #                data_sub['Back/FrontRatio'] = self.backRatio
        #                df = pd.DataFrame.from_dict(data_sub)
                outputvalue=[2017, int(month), int(day), int(hour), int(minute), dni, dhi, ambient_temperatures[rl], wind_speeds[rl]]
                outputvalue+=OTFDict['Wm2']
                sw.writerow(outputvalue)
        except:
            print "Warning: sun altitude below zero at month/day " + str(int(month)) + "/" + str(int(day)) + " time " + str(my_hour)
            pass

print "FINISHED FINISHED"


In [ ]:
#SETUP 9
# Simple example system using Radiance.
#point to an empty directory or existing Radiance directory
writefiletitle="C:\Users\Silvana\Documents\GitHub\\bifacialvf\\bifacialvf\data\Output\Setup9_Rad.csv"
#123 CHECK DATESTART?
datestart = '2017-11-29 9:49:00'    #  Ignoring data from 10/9 and 10/10 from SNOW
dateend = '2017-12-8 9:00:00'    #  Ignoring data from 10/9 and 10/10 from SNOW
rtr=1.2192
C=0.24384
beta = 30
    
testfolder = r'C:\Users\silvana\Documents\RadianceScene\Test'
SRRLtoread = "C:\Users\Silvana\Documents\GitHub\\bifacialvf\\bifacialvf\data\SRRL_data.csv"
demo = RadianceObj('mini_panel',testfolder)  
demo.setGround(0.62) # input albedo or material name like 'concrete'
#   epwfile = demo.getEPW(37.5,-77.6) #can't run this within NREL firewall.  BOO
metdata = demo.readEPW('EPWs\USA_VA_Richmond.Intl.AP.724010_TMY.epw')
# sky data for index 4010 - 4028 (June 17)  
#demo.gendaylit(metdata,4020)

data = readSRRL(SRRLtoread, decimate = True, decimatestyle = "Mean", decimatevalue = 5 )

# Masking data to only do values we are interested in
mask = (data.index > datestart) & (data.index <= dateend )
data=data.loc[mask]
data=data.dropna()
        
direct_normal = data.loc[:,"Direct"]
diffuse_horizontal = data.loc[:,"Diffuse"]
years=data.loc[:,"Year"]
months=data.loc[:,"Month"]
days=data.loc[:,"Day"]
hours=data.loc[:,"Hour"]
minutes=data.loc[:,"Minute"]
ambient_temperatures=data.loc[:,"Temp"]
wind_speeds=data.loc[:,"Wind"]

noRows=len(direct_normal)         
#    SRRL = ({'Month': month_all, 'Day': day_all, 'Year': year_all, 'Hour': hour_all, 'Minute': minute_all,
#                 'Global': global_all, 'Direct': direct_all, 'Diffuse': diffuse_all, 
#                 'Temp': temp_all, 'Humidity': relhum_all, 'Wind': wind_all})
    
with open (writefiletitle,'w') as csvfile:
    sw = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)

    outputtitles=['Year', 'Month', 'Day', 'Hour', 'Minute', 'DNI', 'DHI', 'Tamb', 'VWind',
                   'IMT1_Rad', 'IMT2_Rad', 'IMT3_Rad' ,'IMT4_Rad', 'IMT5_Rad', 'IMT6_Rad']
    sw.writerow(outputtitles)

    for rl in range (0, len(data)):
    
        hour=hours[rl]
        minute=minutes[rl]
        day=days[rl]
        month=months[rl]
        year=years[rl]

        my_hour=round((float(hour)+float(minute)/60),2)

        dni= direct_normal[rl]
        dhi= diffuse_horizontal[rl]

        if dni<0:
            dni=0
        if dhi<0:
            dhi=0

        try:
            with HiddenPrints():
                demo.gendaylitSil(month,day, my_hour, dni, dhi)
                #demo.genCumSky(demo.epwfile)
                # create a scene using monopanel in landscape at 10 deg tilt, 1.5m pitch
                sceneDict = {'tilt':beta,'pitch':rtr,'height':C,'orientation':'landscape','azimuth':180}
                scene = demo.makeScene('mini_panel',sceneDict)
                filelist = demo.getfilelist() #filelist now generated in this function
                octfile = demo.makeOct(filelist) 
                analysis = AnalysisObj(octfile, demo.basename)
                OTFDict=analysis.analysisSil(octfile, demo.basename, scene.frontscan, scene.backscan)    

                myfrontlpts=["0 0.0935567365362 0.01 0 0 1\n0 0.187113473072 0.01 0 0 1\n0 0.280670209608 0.01 0 0 1\n0 0.374226946145 0.01 0 0 1\n0 0.467783682681 0.01 0 0 1\n0 0.561340419217 0.01 0 0 1\n0 0.654897155753 0.01 0 0 1\n0 0.748453892289 0.01 0 0 1\n0 0.842010628825 0.01 0 0 1"]
                data_sub = {key:OTFDict[key] for key in ['x', 'y', 'z', 'Wm2', 'mattype']}

            if OTFDict is not None:                
        #                self.Wm2Front = data_sub.pop('Wm2')
        #                data_sub['Wm2Front'] = self.Wm2Front
        #                self.Wm2Back = backR['Wm2']
        #                data_sub['Wm2Back'] = self.Wm2Back
        #                self.backRatio = [x/(y+.001) for x,y in zip(backR['Wm2'],data['Wm2'])] # add 1mW/m2 to avoid dividebyzero
        #                data_sub['Back/FrontRatio'] = self.backRatio
        #                df = pd.DataFrame.from_dict(data_sub)
                outputvalue=[2017, int(month), int(day), int(hour), int(minute), dni, dhi, ambient_temperatures[rl], wind_speeds[rl]]
                outputvalue+=OTFDict['Wm2']
                sw.writerow(outputvalue)
        except:
            print "Warning: sun altitude below zero at month/day " + str(int(month)) + "/" + str(int(day)) + " time " + str(my_hour)
            pass
        

print "FINISHED FINISHED"
